基于**Learning both Weights and Connections for Efficient Neural Network**论文的stylegan模型修剪

In [6]:
import pandas as pd
import dnnlib
import dnnlib.tflib as tflib
from training import misc
import numpy as np

import tensorflow as tf
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 程序最多只能占用指定gpu90%的显存
#config.gpu_options.allow_growth = True      #程序按需申请内存
#sess = tf.Session(config = config)

tflib.init_tf()
#path = './results/00045-pgan-cifar10_wm-preset-v2-2gpus-fp32'
path = './model'
G, D, Gs  = misc.load_pkl(path+'/sgan_wm_6229.pkl')
#_G, _D, Gs  = misc.load_pkl('./results/00014-sgan-myimagedir_WM-2gpu/network-snapshot-015000.pkl')
print(type(Gs))
#Gs.print_layers()

<class 'dnnlib.tflib.network.Network'>


In [2]:
#print(type(G.vars))
#print(G.vars)
for item in G.vars:
    print(item)
w_avg = Gs.get_var('G_synthesis/4x4/Conv/weight')
#print(w_avg)
#Gs.set_var('G_synthesis/4x4/Conv/weight', w_avg)

lod
dlatent_avg
G_synthesis/noise0
G_synthesis/noise1
G_synthesis/noise2
G_synthesis/noise3
G_synthesis/noise4
G_synthesis/noise5
G_synthesis/noise6
G_synthesis/4x4/Const/const
G_synthesis/4x4/Conv/weight
G_synthesis/4x4/Conv/mod_weight
G_synthesis/4x4/Conv/mod_bias
G_synthesis/4x4/Conv/noise_strength
G_synthesis/4x4/Conv/bias
G_synthesis/4x4/ToRGB/weight
G_synthesis/4x4/ToRGB/mod_weight
G_synthesis/4x4/ToRGB/mod_bias
G_synthesis/4x4/ToRGB/bias
G_synthesis/8x8/Conv0_up/weight
G_synthesis/8x8/Conv0_up/mod_weight
G_synthesis/8x8/Conv0_up/mod_bias
G_synthesis/8x8/Conv0_up/noise_strength
G_synthesis/8x8/Conv0_up/bias
G_synthesis/8x8/Conv1/weight
G_synthesis/8x8/Conv1/mod_weight
G_synthesis/8x8/Conv1/mod_bias
G_synthesis/8x8/Conv1/noise_strength
G_synthesis/8x8/Conv1/bias
G_synthesis/8x8/ToRGB/weight
G_synthesis/8x8/ToRGB/mod_weight
G_synthesis/8x8/ToRGB/mod_bias
G_synthesis/8x8/ToRGB/bias
G_synthesis/16x16/Conv0_up/weight
G_synthesis/16x16/Conv0_up/mod_weight
G_synthesis/16x16/Conv0_up/mod

In [3]:
import dnnlib.tflib.tfutil as tfutil
import numpy as np
def threshold_calculation(w_conv, layer_name,prune_rate):
   # w_conv = item[1] # weight四维参数
    w_parameter = sorted(abs(w_conv.flatten()))
    print(layer_name)
#     print('参数裁剪前总和',sum(w_parameter))
#     print('参数个数',len(w_parameter))
#     print('裁剪个数',int(len(w_parameter) * prune_rate))
    threshold = w_parameter[int(len(w_parameter) * prune_rate)]
    print('阈值',threshold)
    return threshold
    

def prune_main(G, D, Gs,prune_rate):
    print('---------',prune_rate,'---------')
    dict_Gs = Gs.__getstate__()
    dict_variables = G.vars#dict_Gs['variables']
    
    for item in dict_variables:

        #if (item.find('Conv') == -1 or item.find('up') != -1):
        if (item.find('Conv') == -1):
            continue    

        if '/weight' in item:              
            w_conv = Gs.get_var(item) # weight四维参数
            threshold = threshold_calculation(w_conv , item, prune_rate)

            for item1 in range(len(w_conv)):
                for item2 in range(len(w_conv[0])):
                    for item3 in range(len(w_conv[0][0])):
                        for item4 in range(len(w_conv[0][0][0])):
                            if abs(w_conv[item1][item2][item3][item4]) < threshold:
                                w_conv[item1][item2][item3][item4] = 0
           # w_parameter = sorted(abs(w_conv.flatten()))  
            Gs.set_var(item, w_conv)
            G.set_var(item, w_conv)
#             print('参数裁剪后总和',sum(w_parameter))
        if '/mod_weight' in item:              
            w_conv = Gs.get_var(item)
            threshold = threshold_calculation(w_conv,item, prune_rate)

            for item1 in range(len(w_conv)):
                for item2 in range(len(w_conv[0])):
                    if abs(w_conv[item1][item2]) < threshold:
                        w_conv[item1][item2] = 0    
#             print('参数裁剪后总和',sum(w_parameter))           

            
    pkl = os.path.join('model/','test_%d.pkl'%(prune_rate*100))
    misc.save_pkl((G, D, Gs), pkl)  
    print('--------模型model/','test_%d.pkl'%(prune_rate*100),'保存完毕--------')

In [4]:
prune_rates = [0.1,0.3,0.5,0.7,0.9] #修剪的概率
#prune_rates = [0.1,0.9] 
for prune_rate in prune_rates:
    prune_main(G, D, Gs,prune_rate)

--------- 0.1 ---------
G_synthesis/4x4/Conv/weight
阈值 0.16034868
G_synthesis/4x4/Conv/mod_weight
阈值 0.1457323
G_synthesis/8x8/Conv0_up/weight
阈值 0.16349177
G_synthesis/8x8/Conv0_up/mod_weight
阈值 0.17105255
G_synthesis/8x8/Conv1/weight
阈值 0.1642964
G_synthesis/8x8/Conv1/mod_weight
阈值 0.17991652
G_synthesis/16x16/Conv0_up/weight
阈值 0.16491452
G_synthesis/16x16/Conv0_up/mod_weight
阈值 0.20195141
G_synthesis/16x16/Conv1/weight
阈值 0.16372205
G_synthesis/16x16/Conv1/mod_weight
阈值 0.18759973
G_synthesis/32x32/Conv0_up/weight
阈值 0.16382205
G_synthesis/32x32/Conv0_up/mod_weight
阈值 0.19815357
G_synthesis/32x32/Conv1/weight
阈值 0.15526974
G_synthesis/32x32/Conv1/mod_weight
阈值 0.16001329
--------模型model/ test_10.pkl 保存完毕--------
--------- 0.3 ---------
G_synthesis/4x4/Conv/weight
阈值 0.4906885
G_synthesis/4x4/Conv/mod_weight
阈值 0.44419622
G_synthesis/8x8/Conv0_up/weight
阈值 0.50030506
G_synthesis/8x8/Conv0_up/mod_weight
阈值 0.5253031
G_synthesis/8x8/Conv1/weight
阈值 0.50499505
G_synthesis/8x8/Conv1/mod

In [15]:
import os
#os.system("python run_metrics.py --data-dir=./datasets --network=./model/%s.pkl   --metrics=fid50k  --dataset=cifar10_wm3000_mullabel --num-gpus 4" 
 #         % ("test_70"))
os.popen("python run_metrics.py --data-dir=./datasets --network=./model/%s.pkl   --metrics=fid50k  --dataset=cifar10_wm3000_mullabel --num-gpus 4" 
         % ("test_70")).read()


'./datasets\nLocal submit - run_dir: results/00026-run-metrics\ndnnlib: Running run_metrics.run() on localhost...\nEvaluating metrics "fid50k" for "./model/test_70.pkl"...\nSetting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.\nSetting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.\n--------self.tfrecord_dir------ ./datasets/cifar10_wm3000_mullabel\n-----------labels----------------- Tensor("Dataset_1/GatherV2:0", shape=(8, 10), dtype=float32, device=/device:CPU:0)\nTraceback (most recent call last):\n  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1356, in _do_call\n    return fn(*args)\n  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1341, in _run_fn\n    options, feed_dict, fetch_list, target_list, run_metadata)\n  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1429, in _call_tf_sessionrun\n    run_metadat

In [24]:
import subprocess
p = subprocess.Popen('ls -a', stdout=subprocess.PIPE, shell=True)
print(p.stdout.read())

b'.\n..\n.gitignore\n.ipynb_checkpoints\n.stylegan2-cache\nDockerfile\nImage_quality_assessment.ipynb\nLICENSE.txt\nREADME.md\nREADME.pdf\n__pycache__\nanime_generation.ipynb\nanime_generation1.ipynb\ndataset_tool.py\ndatasets\ndnnlib\ndocs\nlabel_error_check.ipynb\nmetrics\nmodel\npretrained_networks.py\nprojector.py\nprune_model.ipynb\nprune_model_\xe5\xa4\x87\xe4\xbb\xbd.ipynb\nreals.png\nresults\nrun_generator.py\nrun_generator1.py\nrun_metrics.py\nrun_projector.py\nrun_training.py\ntest_nvcc.cu\ntraining\n'
